# 1. Required packaged

In [ ]:
'''
pip install panel==0.12.4 bokeh==2.4.1 hvplot==0.7.3 holoviews==1.14.6 pandas==1.3.1
'''

# 2. Panel app

Panel is an open-source Python library that lets you create custom interactive web apps and dashboards by connecting user-defined widgets to plots, images, tables, or text.

Compared to other approaches, Panel is novel in that it supports nearly all plotting libraries, works just as well in a Jupyter notebook as on a standalone secure web server, uses the same code for both those cases, supports both Python-backed and static HTML/JavaScript exported applications, and can be used to develop rich interactive applications without tying your domain-specific code to any particular GUI or web tools.

Panel makes it simple to make:

    Plots with user-defined controls

    Property sheets for editing parameters of objects in a workflow

    Control panels for simulations or experiments

    Custom data-exploration tools

    Dashboards reporting key performance indicators (KPIs) and trends

    Data-rich Python-backed web servers

    and anything in between


In [ ]:
'''
Required librairies
'''

import pandas as pd
import hvplot.pandas
import panel as pn


In [ ]:
#Display config
#Display stretching
pn.extension(sizing_mode="stretch_width")


#Palette Colors
PALETTE = [
    "#ff6f69",
    "#ffcc5c",
    "#88d8b0",
]

#Data: From API
ACCENT_BASE_COLOR = PALETTE[0]
URL = "https://data.seattle.gov/api/views/65db-xm6k/rows.csv?accessType=DOWNLOAD"
LOCAL_FILE = "rows.csv"


#in main page text:

INFO = f"""\
## Widgets as arguments in your Pandas pipeline!
"""

#In Side bar text:

SOURCE_INFO = f"""\
## Data

The data presented are of 5 different companies - Microsoft, Apple, Tesla, Google and Amazon, collected from Tiingo API https://www.tiingo.com.
"""


# Data
try:
    seattle_bikes = pd.read_csv(LOCAL_FILE, parse_dates=["Date"]).set_index("Date")
except Exception:
    seattle_bikes = pd.read_csv(URL, parse_dates=["Date"]).set_index("Date")
    seattle_bikes.to_csv(LOCAL_FILE)



In [ ]:
# Widgets
resample = pn.widgets.Select(value="D", options=["D", "W", "M"], name="Sampling Frequency")
window = pn.widgets.IntSlider(value=50, start=10, end=100, name="Rolling Window Length")
center = pn.widgets.Checkbox(value=True, name="Center")
win_type = pn.widgets.Select(value="gaussian", options=[None, "gaussian"], name="Window Type")
std = pn.widgets.IntSlider(value=10, start=5, end=20, name="std")
line_width = pn.widgets.IntSlider(value=6, start=1, end=20, name="Line Width")



In [ ]:
# A Pandas Dataframe made .interactive with hvPlot
pipeline = (
    seattle_bikes.interactive()
    .resample(resample)
    .sum()
    .rolling(window, center=center, win_type=win_type)
    .sum(std=std)
    .dropna()
)



In [ ]:
# Interactive Plot
plot = pn.panel(
    pipeline.hvplot(
        responsive=True,
        color=PALETTE,
        line_width=line_width,
        yformatter="%.0f",
    ).holoviews().opts(legend_position='top_left'),
    sizing_mode="stretch_both",
    name="Plot",
)



In [ ]:
# Interactive Table
table = pipeline.pipe(
    pn.widgets.Tabulator,
    pagination="remote",
    page_size=20,
    theme="fast",
    sizing_mode="stretch_both",
).panel(name="Table")



In [ ]:
# Layout
tabs = pn.layout.Tabs(plot, table, margin=(10, 25), sizing_mode="stretch_both")
panel_logo = pn.pane.JPG(
            "https://upload.wikimedia.org/wikipedia/commons/7/76/Banco_de_Jos%C3%A9_Bunster_%2830203401352%29.jpg",
            link_url="https://www.linkedin.com/in/joseph-bunster/",
            embed=False,
            sizing_mode="fixed",
            height=100,
            margin=(10,10),
            align="start",
        )

pandas = pn.pane.HTML("<div style='font-size: 15px;text-align: center'>Built with  ❤️ by Joe Bunster</div>")


#Redefining the template: Title
pn.template.FastListTemplate(
    site="Optimal Transport",
    title="Joey Bunster's Time-Series Regression via Optimal Transport Maps ",
    sidebar=[SOURCE_INFO, "## Pandas Pipeline", resample, window, center, win_type, std, "## Plot", line_width, pn.Spacer(height=150), pn.Row(panel_logo), pandas],
    main=[INFO, tabs],
    accent_base_color=ACCENT_BASE_COLOR,
    header_background=ACCENT_BASE_COLOR,
).servable()

# 3. Serve

In [ ]:
'''
panel serve hvplot_interactive.py
'''